# 실시간 Lip Landmark 추출


---

- person: 사람 A~ (박희지, 신은화, 최영윤, 최희수, 한우정)
- num: 반복횟수(00, 01, 02)
- sentence: 00부터 notion에 있는대로

---

```실험 전```
- 사람&문장&반복횟수 **초기화**
----

```실험 동안```
- 코드 실행
- 코드에서 lip localization 한 값들이 뜨기 시작하면 시작
- 입을 벌리면 Frame 별 저장 시작
- 최대한 크게 최대한 가까이 최대한 밝은 공간에서 할 것!
- 얼굴은 정면으로
- 말은 조금은 느리게 (3초에 한단어씩이란 생각으로)
- 웹캠 끌때는 실행되고 있는 웹캠에서 'q' 눌러서 끄기
- 웹캠 반응이 없는데 계속 돌아가고 있다면, 위에 stop 눌러서 꺼주기
- 가끔 웹캠 반응 없으면 웹캠 종료 버튼 

---

```실험후```
- 사진들 저장되어있는거 확인
    - ***********No Detected******** 조심
- 앞이랑 뒤에 상관없는 사진들도 저장이 되었다면 삭제(입을 벌리기 직전 다물기 직전까지만 잘라서 저장: 수작업)
- crop사진들 확인 -> 얼굴 사진 저장되어있는 거랑 매치 시켜서 같이 저장되어있다면 **성공**
- 그렇지 않다면 **다시 찍기** -> 사진들 전부 폐기
- 결과 값 csv 파일로 저장

-  **커널 다시 돌리기**

In [1]:
import pandas as pd
import dlib
import cv2
import numpy as np
import os

In [2]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

In [3]:
def get_rect(shape):
    rw = 0
    rh = 0
    rx = 65535
    ry = 65535
    for (x,y) in shape:
        rw = max(rw,x) # w
        rh = max(rh,y) # h
        rx = min(rx,x) # x
        ry = min(ry,y) # y
    return (rx,ry,rw-rx,rh-ry)

In [4]:
######################################
# person & sentences
person = 'A' 
sentence= '00'
num = '01' 
#######################################
count = 0
dataset = {}
dataset[person] = {}
dataset[person][sentence] ={}
dataset[person][sentence][num] ={}

# create face detector, predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat/shape_predictor_68_face_landmarks.dat") # predict 해주는애

# web cam start
video_capture = cv2.VideoCapture(0) 

while True:
    ret, frame = video_capture.read() #Get Frame: ret True, no: False

   # resize the video
    image = cv2.resize(frame, dsize=(640, 480), interpolation=cv2.INTER_AREA)
    img = image.copy()
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Get faces (up-sampling=1)
    face_detector = detector(img_gray, 1)
    
    if len(face_detector) == 0:
        print("***************No face Detected*************,%d", int(video_capture.get(1)))
    
    # for loop for faces
    for face in face_detector:
        # make prediction and transform to numpy array
        landmarks = predictor(image, face)  # 얼굴에서 68개 점 찾기
        landmarks = shape_to_np(landmarks)
        lip_landmarks = [landmarks[x] for x in range(48, 68)]
        image_crop = image.copy()
        (x, y, w, h) = get_rect(lip_landmarks)
        crop_image = image_crop[y: y+h, x:x+w]
        
        for (i, (x_i, y_i)) in enumerate(lip_landmarks):
            cv2.circle(image, (x_i, y_i), 1, (255, 0, 0), -1)
            
        (x, y, w, h) = get_rect(lip_landmarks)
        
        for s in lip_landmarks:
            s[0] -= x
            s[1] -= y
    
        
    cv2.imshow('result', image)
    
    # exit camera
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    dataset[person][sentence][num][count] ={}   
    if (int(video_capture.get(1)) % 3 == 0): # after 5 sec passed (5초 단위)
        path = 'korea_data/%s/%s/%s/image'%(person, sentence, num)
        crop_path = 'korea_data/%s/%s/%s/crop'%(person, sentence, num)
        
        #path 없다면 만들고 있다면 pass
        try:
            os.makedirs(path)
            os.makedirs(crop_path)
        except FileExistsError as e:
            pass
        img_name = f'img_{count}.jpg'
        mouth_name = f'mouth_{count}.jpg'
        img_path = path + '/' + img_name
        mouth_path = crop_path + '/' + mouth_name
        # save images by frame
        cv2.imwrite(img_path, img)
        cv2.imwrite(mouth_path, crop_image) # 확인용
        print(lip_landmarks)
        print('Saved frame number : ' + str(int(video_capture.get(1)))+ "\n")
        dataset[person][sentence][num][count]= lip_landmarks  
        
        count += 1
        
        
        # exit camera
    if cv2.waitKey(1) == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

[array([ 0, 13]), array([13,  4]), array([25,  0]), array([34,  3]), array([44,  1]), array([55,  6]), array([66, 15]), array([56, 27]), array([45, 31]), array([34, 32]), array([24, 31]), array([12, 25]), array([ 7, 13]), array([25, 10]), array([34, 12]), array([44, 11]), array([59, 15]), array([44, 18]), array([34, 19]), array([25, 18])]
Saved frame number : 3156219

[array([ 0, 11]), array([13,  3]), array([25,  0]), array([34,  3]), array([44,  1]), array([56,  5]), array([67, 14]), array([56, 26]), array([45, 31]), array([34, 33]), array([24, 31]), array([12, 25]), array([ 7, 12]), array([25, 10]), array([34, 11]), array([44, 10]), array([60, 13]), array([44, 18]), array([34, 19]), array([25, 17])]
Saved frame number : 3156225

[array([ 0, 11]), array([13,  4]), array([25,  0]), array([34,  2]), array([44,  0]), array([55,  5]), array([66, 13]), array([56, 26]), array([45, 30]), array([35, 31]), array([25, 29]), array([12, 24]), array([ 7, 12]), array([25, 10]), array([34, 11]), ar

In [5]:
#dataset

In [5]:
# csv 파일은 문장 폴더 안에 입모양 반복 횟수대로 이름지어서 저장됨
fileCount = 0
shapes = []
for count in dataset[person][sentence][num].keys():
    shape = dataset[person][sentence][num][count]
    for (x,y) in shape:
        shapes.append((num,count,x,y))

df = pd.DataFrame(data = shapes,columns = ['num','frame','x','y'])
path = "csv"
try:
    os.makedirs(path)
except FileExistsError as e:
    pass
filename = "%s_%s_%s.csv" %(person, sentence, num)
csvPath = "%s/%s" % (path,filename)
print("saving %s" % csvPath)
df.to_csv(csvPath)

saving csv/A_00_01.csv
